In [0]:
# mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# get git repo
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages
!rm -rf /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab
!git clone https://github.com/pmcgrath249/DeepCV_Lab.git

In [0]:
# move data
!mkdir /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/data
!cp /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/data/data.tfrecord /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/data/data.tfrecord

In [0]:
# permanently change dir
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab

# install waymo dataset utils in utils; https://github.com/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial.ipynb
!cd utils && rm -rf waymo-od > /dev/null
!cd utils && git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd utils/waymo-od && git branch -a
!cd utils/waymo-od && git checkout remotes/origin/r1.0
!pip3 install --upgrade pip
!pip3 install waymo-open-dataset

# install requirements
!cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/ && pip3 install -r requirements.txt

# install own package
!cd /content/drive/My\ Drive/Colab\ Notebooks/DeepCV_Packages/DeepCV_Lab/ && pip3 install -e .

# directly retrieve data from google
# https://medium.com/@philipplies/transferring-data-from-google-drive-to-google-cloud-storage-using-google-colab-96e088a8c041
'''
from google.colab import auth
auth.authenticate_user()
project_id = 'nifty-depth-246308'
!gcloud config set project {project_id}
!gsutil ls
bucket_name = 'medium_demo_bucket_190710'
!gsutil -m cp -r /content/mnt/My\ Drive/Colab\ Notebooks/DeepCV_Packages/* gs://{bucket_name}/
'''